Capstone Project

In [1]:
import pandas as pd
import numpy as np

In [4]:
range (9)

range(0, 9)

In [5]:
L = range (9)

In [6]:
L

range(0, 9)

In [7]:
type (L)

range

In [18]:
L = [1, 2, 3, 4]
len (L)

4

In [11]:
K = L[0] > 2

In [12]:
K

False

In [22]:
def elementwise_greater_than(L, thresh):
    W = []
    for i in range (len (L)):
        K = L[i]> thresh
        W.append (K)
    return W

In [23]:
J = elementwise_greater_than([1,2,3,4], 2)

In [24]:
J

[False, False, True, True]

In [25]:
print ('hello', 'pluto', end = '')

hello pluto

In [26]:
p = 9

In [27]:
k = str(p)

In [28]:
type (k)

str

In [29]:
help (dict)

Help on class dict in module builtins:

class dict(object)
 |  dict() -> new empty dictionary
 |  dict(mapping) -> new dictionary initialized from a mapping object's
 |      (key, value) pairs
 |  dict(iterable) -> new dictionary initialized as if via:
 |      d = {}
 |      for k, v in iterable:
 |          d[k] = v
 |  dict(**kwargs) -> new dictionary initialized with the name=value pairs
 |      in the keyword argument list.  For example:  dict(one=1, two=2)
 |  
 |  Methods defined here:
 |  
 |  __contains__(self, key, /)
 |      True if D has a key k, else False.
 |  
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(...)
 |      x.__getitem__(y) <==> x[y]
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __init__(self, /, *args, **kwargs)
 |

In [30]:
D = {'one': 1, 'second': 2, 'third': 3}

In [32]:
J = D.keys

In [33]:
J

<function dict.keys>

In [34]:
D.values ()

dict_values([1, 2, 3])

In [35]:
D.keys ()

dict_keys(['one', 'second', 'third'])

In [36]:
D ['second']

2

In [37]:
D[-1]

KeyError: -1

In [38]:
D{-1}

SyntaxError: invalid syntax (<ipython-input-38-e90777912426>, line 1)

In [39]:
D[0]

KeyError: 0

In [40]:
L = ['one', 'second','third']

In [41]:
N = [1,2,3]

In [42]:
X = zip (L, N)

In [43]:
X

In [44]:
X[0]

TypeError: 'zip' object is not subscriptable

In [45]:
D{}

SyntaxError: invalid syntax (<ipython-input-45-1444618df18e>, line 1)

In [47]:
c = 'it\'s ok'
len (c)

7

In [48]:
d = """hey"""
len (d)

3

In [49]:
e = '\n'
len (e)

1

In [50]:
zip_code='1234x'

In [51]:
zip_code.isdigit ()

False